In [ ]:
import os
from openai import OpenAI
import rich
from google.colab import userdata
from google.colab import drive

Note: we need to use models that support reasoning like o1 and o3-mini

In [ ]:
api_key = userdata.get('OPENAI_API_KEY')
MODEL = "o3-mini"

openai = OpenAI(api_key=api_key)

**Advanced reasoning and problem-solving models**

# Chat Completion API

https://platform.openai.com/docs/guides/reasoning?api-mode=chat

Reasoning model takes time to respond.

In [ ]:
prompt = """
Write a bash script that takes a matrix represented as a string with
format '[1,2],[3,4],[5,6]' and prints the transpose in the same format.
"""

response = openai.chat.completions.create(
    model=MODEL,
    reasoning_effort="medium",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ]
)

print(rich.print(response.usage))
print(response.choices[0].message.content)

CompletionUsage(
    completion_tokens=2384,
    prompt_tokens=44,
    total_tokens=2428,
    completion_tokens_details=CompletionTokensDetails(
        accepted_prediction_tokens=0,
        audio_tokens=0,
        reasoning_tokens=1472,
        rejected_prediction_tokens=0
    ),
    prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
)

None
Below is one acceptable solution. Save the script as, for example, transpose.sh, give it executable permissions (chmod +x transpose.sh), then run it passing the matrix string as its first argument.

Below is the complete script:

------------------------------------------------
#!/bin/bash
# Check that we got exactly one argument.
if [ "$#" -ne 1 ]; then
  echo "Usage: $0 '[1,2],[3,4],[5,6]'"
  exit 1
fi

matrix_str="$1"

# Convert the single input string into multiple lines (one per row) by:
# • Removing any leading "[" and trailing "]"
# • Replacing "],[" with a newline.
rows=($(echo "$matrix_str" | sed 's/^\[//; s/\]$//; s/\],\[/\n/g'))

# Number of rows in the matrix.
num_rows=${#rows[@]}

# Determine the number of columns from the first row.
IFS=',' read -ra first_row <<< "${rows[0]}"
num_cols=${#first_row[@]}

# Create an array to hold the transposed rows.
declare -a transposed_rows

# Loop over each column index (which will become rows in the transpose)
for (( col=0; col<nu

# Responses API

https://platform.openai.com/docs/guides/reasoning?api-mode=responses

In [ ]:
prompt = """
Write a bash script that takes a matrix represented as a string with
format '[1,2],[3,4],[5,6]' and prints the transpose in the same format.
"""

response = openai.responses.create(
    model=MODEL,
    reasoning={"effort": "medium"}, # reasoning_effort now become reasoning and accept a dictionary
    input=[
        {
            "role": "user",
            "content": prompt
        }
    ]
)

rich.print(response.usage)
print(response.output_text)

ResponseUsage(
    input_tokens=103,
    input_tokens_details=InputTokensDetails(cached_tokens=0),
    output_tokens=1880,
    output_tokens_details=OutputTokensDetails(reasoning_tokens=1152),
    total_tokens=1983
)

Below is one solution in a bash script. Save it, for example, as transpose_matrix.sh, give it execute permission (chmod +x transpose_matrix.sh), and then run it with the matrix string as its only argument.

#!/bin/bash
# This script expects a single argument in the form:
#   "[1,2],[3,4],[5,6]"
# It will compute the transpose and output it in the same format.
#
# For example:
#   ./transpose_matrix.sh "[1,2],[3,4],[5,6]"
# will output:
#   [1,3,5],[2,4,6]

if [ "$#" -ne 1 ]; then
  echo "Usage: $0 \"[row1],[row2],...\""
  exit 1
fi

input="$1"
# Remove any spaces from the string (optional)
input=$(echo "$input" | tr -d ' ')

# Convert the input string into rows.
# We replace the "],[" separator with a space and then remove the remaining brackets.
# After this, each row is a comma-separated list of elements.
rows=$(echo "$input" | sed 's/\],\[/ /g; s/^\[//; s/\]$//; s/\[//g; s/\]//g')

# Read the rows into an array.
i=0
declare -a matrix
for row in $rows; do
    matrix[$i]="$row"
    i=